# V28: Current status summary (TOI-5807.01)

This notebook consolidates the current state of the analysis into a compact summary:
- headline TRICERATOPS(+ ) results (fast + higher-fidelity, with/without AO constraints)
- depth and radius ranges implied by the sensitivity runs
- remaining warnings that are still unresolved with the current data

It does not introduce new analysis; it only reads previously-generated artifacts.


In [ ]:
from __future__ import annotations

import json
import sys
from pathlib import Path

import numpy as np

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

step_id = '42_v28_current_status_summary'
run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)

def read_json(path: str) -> dict:
    return json.loads(Path(path).read_text())

v22 = read_json('docs/tutorials/artifacts/tutorial_toi-5807-incremental/36_v22_triceratops_fpp/triceratops_fpp.json')
v23 = read_json('docs/tutorials/artifacts/tutorial_toi-5807-incremental/37_v23_triceratops_fpp_with_contrast_curve/triceratops_fpp_with_contrast_curve.json')
v24 = read_json('docs/tutorials/artifacts/tutorial_toi-5807-incremental/38_v24_triceratops_fpp_sensitivity_grid/triceratops_fpp_sensitivity_grid.json')
v25 = read_json('docs/tutorials/artifacts/tutorial_toi-5807-incremental/39_v25_localization_consistency_82_83/localization_consistency_82_83.json')
v26 = read_json('docs/tutorials/artifacts/tutorial_toi-5807-incremental/40_v26_dilution_and_radius_range/dilution_and_radius_range.json')
v27 = read_json('docs/tutorials/artifacts/tutorial_toi-5807-incremental/41_v27_triceratops_standard_with_contrast_curve/triceratops_standard_with_contrast_curve.json')

def pick_fpp(blob: dict) -> dict:
    r = blob.get('fpp_result', {})
    return {
        'depth_ppm_used': blob.get('depth_ppm_used'),
        'fpp': r.get('fpp'),
        'nfpp': r.get('nfpp'),
        'disposition': r.get('disposition'),
        'prob_planet': r.get('prob_planet'),
        'n_nearby_sources': r.get('n_nearby_sources'),
        'sectors_used': r.get('sectors_used'),
    }

rows = v24.get('rows', [])
depths = [float(r['depth_ppm_used']) for r in rows]

def fpp_vals(use_cc: bool) -> list[float]:
    out = []
    for r in rows:
        if bool(r.get('use_contrast_curve')) != bool(use_cc):
            continue
        fpp = r.get('fpp_result', {}).get('fpp')
        if fpp is None:
            continue
        out.append(float(fpp))
    return out

fpp_no = fpp_vals(False)
fpp_cc = fpp_vals(True)

centroid_summary = {}
for sec in ['82', '83']:
    centroid_summary[sec] = {
        'centroid_shift_arcsec': v25[sec]['V08']['metrics'].get('centroid_shift_arcsec'),
        'significance_sigma': v25[sec]['V08']['metrics'].get('significance_sigma'),
        'v09_flags': v25[sec]['V09'].get('flags', []),
    }

warnings: list[str] = []
warnings.append('Depth is preprocessing-dependent (see V24 depth range).')
warnings.append('Difference image (V09) is flagged as unreliable in both sectors (see V25).')

summary = {
    'baseline': {
        'tic_id': int(sh.TIC_ID),
        'toi': str(sh.TOI_LABEL),
        'sectors_used': [82, 83],
        'period_days': float(sh.PERIOD_DAYS),
        't0_btjd': float(sh.T0_BTJD),
        'duration_hours': float(sh.DURATION_HOURS),
        'stellar_radius_rsun': float(sh.STELLAR.radius),
        'stellar_mass_msun': float(sh.STELLAR.mass),
    },
    'tricera': {
        'v22_fast_no_ao': pick_fpp(v22),
        'v23_fast_with_ao': pick_fpp(v23),
        'v27_higher_fidelity_with_ao': pick_fpp(v27),
    },
    'sensitivity': {
        'depth_ppm_range_from_v24': {
            'min': float(min(depths)),
            'median': float(np.median(depths)),
            'max': float(max(depths)),
        },
        'fpp_range_no_ao': {'min': float(min(fpp_no)), 'max': float(max(fpp_no))} if fpp_no else None,
        'fpp_range_with_ao': {'min': float(min(fpp_cc)), 'max': float(max(fpp_cc))} if fpp_cc else None,
    },
    'radius': {
        'planet_radius_rearth_if_on_target_from_v26': v26.get('planet_radius_rearth_if_on_target'),
        'physics_flags_from_v26': v26.get('physics_flags'),
    },
    'localization': {
        'centroid_shift_summary': centroid_summary,
    },
    'remaining_warnings': warnings,
}

json_name = 'current_status_summary.json'
(run_out_dir / json_name).write_text(json.dumps(summary, indent=2, sort_keys=True))
if docs_out_dir is not None:
    (docs_out_dir / json_name).write_text(json.dumps(summary, indent=2, sort_keys=True))

headline = {
    'depth_ppm_range_from_v24': summary['sensitivity']['depth_ppm_range_from_v24'],
    'rp_rearth_if_on_target': summary['radius']['planet_radius_rearth_if_on_target_from_v26'],
    'v22_fpp_nfpp': (summary['tricera']['v22_fast_no_ao']['fpp'], summary['tricera']['v22_fast_no_ao']['nfpp']),
    'v23_fpp_nfpp': (summary['tricera']['v23_fast_with_ao']['fpp'], summary['tricera']['v23_fast_with_ao']['nfpp']),
    'v27_fpp_nfpp': (summary['tricera']['v27_higher_fidelity_with_ao']['fpp'], summary['tricera']['v27_higher_fidelity_with_ao']['nfpp']),
    'remaining_warnings': summary['remaining_warnings'],
}
print(json.dumps(headline, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

A summary JSON is written to:
- `docs/tutorials/artifacts/tutorial_toi-5807-incremental/42_v28_current_status_summary/current_status_summary.json`

Headline values should match earlier steps (V22, V23, V24, V26, V27).

</details>


In [ ]:
import json

import matplotlib.pyplot as plt
import numpy as np

# Simple 3-point FPP comparison (log scale)
labels = ['V22 fast (no AO)', 'V23 fast (AO)', 'V27 higher-fidelity (AO)']
fpps = [
    summary['tricera']['v22_fast_no_ao']['fpp'],
    summary['tricera']['v23_fast_with_ao']['fpp'],
    summary['tricera']['v27_higher_fidelity_with_ao']['fpp'],
]
vals = [float(v) if v is not None else np.nan for v in fpps]

fig, ax = plt.subplots(figsize=(9.5, 3.6), dpi=150)
ax.bar(labels, vals, color=['#1f77b4', '#2ca02c', '#2ca02c'])
ax.set_yscale('log')
ax.set_ylabel('FPP (log)')
ax.set_title('TOI-5807.01: TRICERATOPS(+ ) headline FPP values')
ax.grid(True, which='both', axis='y', alpha=0.2)
for i, v in enumerate(vals):
    if np.isfinite(v):
        ax.text(i, v, f"{v:.1e}", ha='center', va='bottom', fontsize=9)
fig.tight_layout()

png_name = 'V28_status_fpp.png'
run_png = run_out_dir / png_name
fig.savefig(run_png)
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / png_name)
plt.close(fig)

print(json.dumps({'run_plot_path': str(run_png), 'docs_plot_path': str((docs_out_dir / png_name) if docs_out_dir is not None else None)}, indent=2, sort_keys=True))


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/42_v28_current_status_summary/V28_status_fpp.png" width="980" />


<details>
<summary><b>Analysis</b></summary>

- **Statistical validation (TRICERATOPS):** the planet hypothesis is strongly favored under the current baseline (gated sectors 82+83) and AO constraint. The higher-fidelity run (V27) returns `FPP~3e-6` with `NFPP≈0`, which is comfortably below the common `FPP<0.01` validation threshold.
- **What this does and does not imply:** this is a probabilistic screen against EB/blend scenarios (given Gaia neighbors + population priors + the AO contrast curve). It does *not* guarantee the signal is on-target, nor does it rule out all classes of instrumental/systematic false alarms.
- **Planet size (on-target assumption):** mapping the depth range from the sensitivity grid (V24) onto the stellar radius yields `Rp~2.5–2.8 R⊕` if the signal is on the target (V26).
- **Localization (current data):** centroid shifts are modest and not strongly inconsistent with the target, but the difference-image depth map is flagged as unreliable in both sectors (V25). Treat on-target localization as **inconclusive** from difference imaging alone.
- **Remaining unresolved warnings (current data):** the inferred depth still depends on preprocessing choices (V24), and earlier sector-to-sector depth consistency required gating (V21). Any future change to the trusted sector set or detrending baseline should trigger a re-run of depth and FPP.

</details>
